In [1]:
from datetime import datetime, timedelta

In [2]:
class PunchCard(object):
    def __init__(self, name):
        self.name = name
        
        self.hours_required = 42.5
        self.punches = {}
        self.total_hours = 0
        self.days_counted = []
        self.on_clock = False
        
    def calculate_total_hrs(self):
        for day, pnchs in self.punches.items():
            if day not in self.days_counted:
                if len(pnchs) == 2:
                    self.total_hours += (datetime.strptime(pnchs["out"], '%I:%M%p') - datetime.strptime(pnchs["in"], '%I:%M%p')).seconds / 3600
                    self.days_counted.append(day)
                    
    def calculate_hours_expected(self):
        '''how many total hours should I work by the end of today to be up to par with hours required?'''
        hrs_per_day = self.hours_required/5
        return ((hrs_per_day*len(self.punches)) - self.total_hours)
        
    def punch_in(self, dow, time):
        if self.on_clock == False:
            self.punches[dow] = {"in": time}
            self.on_clock = True
        else:
            for day, pnchs in self.punches.items():
                if len(pnchs) == 1 and day != dow:
                    self.edit_punch(dow, 'in', time)
                    print("You didn't punch out on", day)
        
    def punch_out(self, dow, time):
        if self.on_clock == True:
            if dow in self.punches.keys():
                self.punches[dow]["out"] = time
                self.calculate_total_hrs()
                self.on_clock = False
            else:
                print("You didn't clock in on %s!" %dow)
        else:
            print("You aren't on the clock!")
                
    def edit_punch(self, dow, punch, time):
        if dow in self.punches.keys():
            if punch in self.punches[dow].keys():
                self.punches[dow][punch]=time
            
    def when_should_i_leave(self):
        self.calculate_total_hrs()
        if self.on_clock == True:
            for day, pnchs in self.punches.items():
                if len(pnchs) == 1:
                    return (datetime.strptime(pnchs["in"], '%I:%M%p') + timedelta(hours=self.calculate_hours_expected())).strftime("%I:%M %p")
        else:
            return "You aren't on the clock!"

In [3]:
josh = PunchCard('josh')

In [4]:
josh.punch_in('mon', '8:15AM')
josh.punch_out('mon', '4:43PM')

In [5]:
josh.punch_in('tue', '8:26AM')

In [6]:
josh.when_should_i_leave()

'04:58 PM'